In [1]:
print("Hello, World!")

Hello, World!


In [2]:
import os

In [3]:
%pwd  #project working directory

'/Users/apple/Downloads/Project/Wine-Quality-Prediction-End-to-End-Project/research'

In [4]:
os.chdir("../") #change directory to parent,change directory

In [5]:
%pwd

'/Users/apple/Downloads/Project/Wine-Quality-Prediction-End-to-End-Project'

In [6]:
from dataclasses import dataclass  # 🧾 dataclass ব্যবহারে সহজে configuration বা data structure তৈরি করা যায়
from pathlib import Path  # 🗂️ ফাইল ও ডিরেক্টরি পাথ নিয়ে কাজ করার জন্য

# 📦 DataIngestionConfig একটি ইম্মিউটেবল (frozen=True) কনফিগারেশন ক্লাস
@dataclass(frozen=True)
class DataIngestionConfig:
    # 🗂️ মূল ডিরেক্টরি যেখানে ডেটা ইনজেশন সম্পর্কিত সব কিছু থাকবে
    root_dir: Path

    # 🌐 যেখান থেকে ডেটা ডাউনলোড করা হবে (source URL)
    source_URL: str

    # 💾 লোকাল সিস্টেমে কোথায় ZIP ফাইলটি সেভ হবে
    local_data_file: Path

    # 📂 ডেটা আনজিপ করার পর কোন ডিরেক্টরিতে রাখা হবে
    unzip_dir: Path

#config.yaml e ase data ingestion er configuration thakbe

In [7]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])



    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [9]:
import os  # 📁 ফাইল ও ডিরেক্টরি সংক্রান্ত অপারেশনের জন্য

# 🌐 URL থেকে ফাইল ডাউনলোড করার জন্য urllib এর request মডিউল
import urllib.request as request

# 📦 .zip ফাইল এক্সট্রাক্ট করার জন্য
import zipfile

# 📝 প্রজেক্টের কাস্টম লগার – লগ মেসেজ জেনারেট করার জন্য
from mlProject import logger

# 📏 ফাইল সাইজ বের করার ইউটিলিটি ফাংশন
from mlProject.utils.common import get_size


In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        # ⚙️ কনফিগারেশন অবজেক্ট সেট করা হচ্ছে (যেখানে path ও URL আছে)
        self.config = config

    def download_file(self):
        # 🌐 যদি লোকাল ফাইল আগে থেকে না থাকে, তাহলে ডাউনলোড করা হবে
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            # ✅ সফলভাবে ডাউনলোড হলে লগে দেখানো হবে
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            # 📁 ফাইল আগে থেকেই আছে, তাহলে শুধু সাইজ দেখাবে
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        📦 `.zip` ফাইল আনজিপ করে নির্ধারিত ফোল্ডারে এক্সট্রাক্ট করা হয়
        """
        unzip_path = self.config.unzip_dir

        # 📂 আনজিপ করার ডিরেক্টরি যদি না থাকে, তাহলে তৈরি করো
        os.makedirs(unzip_path, exist_ok=True)

        # 🔓 zip ফাইল খুলে এক্সট্রাক্ট করে নির্দিষ্ট ফোল্ডারে
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Extracted zip file to: {unzip_path}")


In [14]:
try:
    # 🛠️ কনফিগারেশন ম্যানেজার কল করে কনফিগ অবজেক্ট তৈরি করা হচ্ছে
    config = ConfigurationManager()

    # 📦 ডেটা ইনজেশন সংক্রান্ত কনফিগ সংগ্রহ
    data_ingestion_config = config.get_data_ingestion_config()

    # 🍽️ DataIngestion অবজেক্ট তৈরি
    data_ingestion = DataIngestion(config=data_ingestion_config)

    # 🌐 ডেটা ডাউনলোড করা হচ্ছে
    data_ingestion.download_file()

    # 📂 ডেটা আনজিপ করে এক্সট্রাক্ট করা হচ্ছে
    data_ingestion.extract_zip_file()

except Exception as e:
    # ❌ যদি কোনো এরর আসে তাহলে সেটা রেইজ করা হবে
    raise e


[2025-05-16 09:58:36,031: INFO: common: yaml file: config/config.yaml loaded successfully ✅]
[2025-05-16 09:58:36,082: INFO: common: yaml file: params.yaml loaded successfully ✅]
[2025-05-16 09:58:36,098: INFO: common: yaml file: schema.yaml loaded successfully ✅]
[2025-05-16 09:58:36,103: INFO: common: ✅ ডিরেক্টরি তৈরি হয়েছে: artifacts]
[2025-05-16 09:58:36,207: INFO: common: ✅ ডিরেক্টরি তৈরি হয়েছে: artifacts/data_ingestion]
[2025-05-16 09:58:37,782: INFO: 3811897134: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: C4C3:32E2D1:9F0369:B20E62:6826B6F9
Accept-Ranges: bytes
Dat